In [1]:
import re
import pandas as pd
import re

# defining string
hcmv_fasta = ""
#read in fasta file into a list
with open('/Users/jengs/OneDrive - Oregon Health & Science University/From_Box/Pre-Share/PPG/scRNA_20250128/genomes_practice/HCMV.fa') as f:
        hcmv_fasta_list= f.readlines()
#remove first element as this is the seq name
hcmv_fasta_list = hcmv_fasta_list[1:]
#create one hcmv string
#remove new line character
for line in hcmv_fasta_list:
        hcmv_fasta += line.rstrip()



In [57]:
#defining substring
stop_codon = "TAA"
stop_codon2 = "TAG"
stop_codon3 = "TGA"
forward_stop_codons=(stop_codon,stop_codon2,stop_codon3)
polyA = "AATAAA"

#reverse strand
reverse_codon = "TTA"
reverse_codon2 = "CTA"
reverse_codon3 = "TCA"
reverse_stop_codons=(reverse_codon,reverse_codon2,reverse_codon3)
reverse_polyA = "TTTATT"


In [58]:
hcmv_gtf=pd.read_csv('/Users/jengs/OneDrive - Oregon Health & Science University/From_Box/Pre-Share/PPG/annotation_2025-04-09/annotation_2025_04_10_Meaghan.txt',sep="\t",header=None)
hcmv_gtf.columns=["seq name","source","feature","start","end","score","strand","frame","attribute"]


In [65]:
try:
        with open('hcmv_meaghan_three_prime_utr.txt','w') as output:
                #output.write("seq name\tsource\tfeature\tstart\tend\tnew_start\tnew_end\tthreePrimeSeq\tscore\tstrand\tframe\tattribute\n")
                for idx,gene_entry in hcmv_gtf.iterrows():
                        strand=gene_entry["strand"]
                        #create a list of stop codon start locations within gene sequence
                        stop_codon_start_list=[]
                        #stop_codon_start_idx=0
                        #create new start and new stop
                        #new start will be start of 3 prime UTR for reverse strand
                        #new stop will be end of 3 prime UTR for forward strand
                        new_start=gene_entry["start"]
                        new_end=gene_entry["end"]
                        polyA_idx=0
                        threePrimeSeq=""
                        #subset fasta with coordinates of transcript
                        gene_fasta=hcmv_fasta[gene_entry["start"]:gene_entry["end"]]
                        if strand=="+":
                                for _, codon in enumerate(forward_stop_codons):
                                        for m in re.finditer(codon, gene_fasta):
                                                stop_codon_start_list.append(m.start(0))
                                #get the max idx of start codon on forward strand
                                if len(stop_codon_start_list)>0:
                                    #search for poly A after stop codon
                                    #add the start of transcript to the stop codon idx to get idx of hcmv genome
                                    #max search at 500 bp
                                    stop_codon_start_idx=gene_entry["start"]+max(stop_codon_start_list)
                                    #polyA_idx will be the new end coordinate
                                    polyA_idx=re.search(polyA,hcmv_fasta[stop_codon_start_idx:(stop_codon_start_idx+500)])
                                    if polyA_idx:
                                        polyA_idx=stop_codon_start_idx+polyA_idx.end(0)
                                    else:
                                        polyA_idx=stop_codon_start_idx+500
                                    new_end=polyA_idx
                                    threePrimeSeq=hcmv_fasta[stop_codon_start_idx:polyA_idx]
    
                        else:
                                rev_polyA_idx=[]
                                for _, rev_codon in enumerate(reverse_stop_codons):
                                    for m in re.finditer(rev_codon, gene_fasta):
                                            stop_codon_start_list.append(m.end(0))
                                #get minimum idx of start codon on reverse strand
                                #search for revPolyA before stop codon
                                #add start of transcript to stop codon idx to get idx of hcmv genome
                                #max search at 500 bp
                                #only change new_start if found stop codon
                                if len(stop_codon_start_list)>0:
                                    stop_codon_start_idx=gene_entry["start"]+min(stop_codon_start_list)
                                    for revPolyA in re.finditer(reverse_polyA,hcmv_fasta[stop_codon_start_idx-500:stop_codon_start_idx]):
                                        rev_polyA_idx.append(revPolyA.start(0))
                                    if len(rev_polyA_idx)>0:
                                        #get the revpolyA closest to the stop codon
                                        polyA_idx=stop_codon_start_idx-500+max(rev_polyA_idx)
                                        new_start=polyA_idx
                                    else:
                                        polyA_idx=stop_codon_start_idx-500
                                        new_start=polyA_idx
                                        
                                    
                                threePrimeSeq=hcmv_fasta[new_start:stop_codon_start_idx]
                        #output new line of gtf with putative 3 prime utr
                        #commenting out extraneous cols used for double checking
                        output.write(gene_entry["seq name"] +"\t")
                        output.write(gene_entry["source"] +"\t")
                        output.write(gene_entry["feature"] +"\t")
                        #output.write(str(gene_entry["start"]) +"\t")
                        #output.write(str(gene_entry["end"]) +"\t")
                        output.write(str(new_start)+"\t")
                        output.write(str(new_end)+"\t")
                        #output.write(threePrimeSeq+"\t")
                        output.write(gene_entry["score"] +"\t")
                        output.write(gene_entry["strand"] +"\t")
                        output.write(gene_entry["frame"] +"\t")
                        output.write(gene_entry["attribute"] +"\n")                        
                
        output.close()
except IOError:
        print("Error")
